In [70]:
import os
import numpy as np
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

##### Data Preparing

In [71]:
x_train = []
y_train = []
x_test = []
y_test = []

cnt = 0
for file  in os.listdir('train/pos'):
    with open('train/pos/' + file, 'r') as f:
        try:
            s = f.read()
            x_train.append(s)
            y_train.append(1)
            cnt += 1
            
        except Exception:
            pass

print('OK')
cnt = 0
for file  in os.listdir('train/neg'):
    with open('train/neg/' + file, 'r') as f:
        try:
            s = f.read()
            x_train.append(s)
            y_train.append(0)

        except Exception:
            pass

print('OK')
cnt = 0
for file  in os.listdir('test/pos'):
    with open('test/pos/' + file, 'r') as f:
        try:
            s = f.read()
            x_test.append(s)
            y_test.append(1)

        except Exception:
            pass

print('OK')
cnt = 0
for file  in os.listdir('test/neg'):
    with open('test/neg/' + file, 'r') as f:
        try:
            s = f.read()
            x_test.append(s)
            y_test.append(0)
        except Exception:
            pass
print('OK')

X_train = np.array(x_train)
Y_train = np.array(y_train)
X_test = np.array(x_test)
Y_test = np.array(y_test)



OK
OK
OK
OK


In [72]:
from keras.utils.data_utils import pad_sequences
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [73]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [74]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_3 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_6 (Activation)   (None, 256)               0         
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [84]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
         validation_split=0.15,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
166/166 [==============================] - 38s 228ms/step - loss: 0.3537 - accuracy: 0.8512 - val_loss: 0.4539 - val_accuracy: 0.7981
Epoch 2/10
166/166 [==============================] - 45s 272ms/step - loss: 0.3412 - accuracy: 0.8572 - val_loss: 0.3586 - val_accuracy: 0.8355


In [85]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [86]:
accr = model.evaluate(test_sequences_matrix,Y_test)

782/782 [==============================] - 24s 31ms/step - loss: 0.3314 - accuracy: 0.8557


In [87]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.331
  Accuracy: 0.856


In [98]:
test_sequences = tok.texts_to_sequences(["This was a dissapointment", "Good choice I liked it so much"])
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
yy_test = np.array([0, 1])

out = model.predict(test_sequences_matrix)


1/1 [==============================] - 0s 38ms/step


In [99]:
accr = model.evaluate(test_sequences_matrix,yy_test)

1/1 [==============================] - 0s 43ms/step - loss: 0.4703 - accuracy: 1.0000
